# Rwanda Trade Data Analysis
## Comprehensive Analysis of Rwanda's External Trade Dataset

This notebook provides a comprehensive analysis of Rwanda's formal external trade in goods, covering exports, imports, and re-exports data from 2023Q1 to 2025Q1.

## Dataset Overview
- **Source**: National Institute of Statistics of Rwanda (NISR)
- **Period**: 2023Q1 - 2025Q1
- **Currency**: US$ millions
- **Sheets**: 13 different sheets covering various trade dimensions
- **Key Metrics**: Trade values, percentages, growth rates, and regional breakdowns

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

Libraries imported successfully!


## 1. Dataset Structure Analysis

Let's first examine the structure of our Excel dataset to understand what data we're working with.

In [2]:
# Load and examine the Excel file structure
excel_file_path = "../data/raw/2025Q1_Trade_report_annexTables.xlsx"

try:
    excel_file = pd.ExcelFile(excel_file_path)
    sheet_names = excel_file.sheet_names
    
    print(f"📊 Excel file loaded successfully!")
    print(f"📋 Total sheets found: {len(sheet_names)}")
    print(f"\n📑 Available sheets:")
    for i, sheet in enumerate(sheet_names, 1):
        print(f"   {i}. {sheet}")
        
except Exception as e:
    print(f"❌ Error loading Excel file: {e}")

📊 Excel file loaded successfully!
📋 Total sheets found: 13

📑 Available sheets:
   1. Graph Overall
   2. Graph EAC
   3. EAC
   4. Total trade with the World
   5. Regional blocks
   6. Trade by continents
   7. Sheet11
   8. ExportCountry
   9. ImportCountry
   10. ReexportsCountry
   11. ExportsCommodity
   12. ImportsCommodity
   13. ReexportsCommodity


In [3]:
# Analyze each sheet's structure
sheet_analysis = {}

for sheet_name in sheet_names:
    try:
        df = pd.read_excel(excel_file_path, sheet_name=sheet_name, header=None)
        sheet_analysis[sheet_name] = {
            'shape': df.shape,
            'columns': df.shape[1],
            'rows': df.shape[0],
            'memory_usage': df.memory_usage(deep=True).sum(),
            'sample_data': df.head(3).to_dict('records')
        }
        print(f"✅ {sheet_name}: {df.shape[0]} rows × {df.shape[1]} columns")
    except Exception as e:
        print(f"❌ {sheet_name}: Error - {e}")
        sheet_analysis[sheet_name] = {'error': str(e)}

print(f"\n📈 Analysis complete for {len(sheet_analysis)} sheets")

✅ Graph Overall: 12 rows × 11 columns
✅ Graph EAC: 13 rows × 11 columns
✅ EAC: 31 rows × 21 columns
✅ Total trade with the World: 17 rows × 12 columns
✅ Regional blocks: 31 rows × 11 columns
✅ Trade by continents: 23 rows × 20 columns
✅ Sheet11: 8 rows × 6 columns
✅ ExportCountry: 31 rows × 13 columns
✅ ImportCountry: 27 rows × 13 columns
✅ ReexportsCountry: 28 rows × 13 columns
✅ ExportsCommodity: 16 rows × 14 columns
✅ ImportsCommodity: 16 rows × 14 columns
✅ ReexportsCommodity: 17 rows × 14 columns

📈 Analysis complete for 13 sheets


## 2. Overall Trade Analysis

Let's start by analyzing the overall trade data from the 'Graph Overall' sheet.

In [4]:
# Load overall trade data
overall_df = pd.read_excel(excel_file_path, sheet_name='Graph Overall', header=None)

# Extract the data (skipping metadata rows)
overall_data = []
quarters = []

# Find the header row with quarters
for i, row in overall_df.iterrows():
    if pd.notna(row.iloc[1]) and 'Flow/Period' in str(row.iloc[1]):
        quarters = [str(q) for q in row.iloc[2:] if pd.notna(q)]
        break

print(f"📅 Found {len(quarters)} quarters: {quarters}")

# Extract trade flows
trade_flows = ['Exports', 'Imports', 'Re-Exports', 'Total Trade', 'Trade Balance']

for flow in trade_flows:
    for i, row in overall_df.iterrows():
        if pd.notna(row.iloc[1]) and flow == str(row.iloc[1]).strip():
            values = []
            for val in row.iloc[2:]:
                try:
                    values.append(float(val) if pd.notna(val) else 0)
                except (ValueError, TypeError):
                    continue
            
            for quarter, value in zip(quarters[:len(values)], values):
                overall_data.append({
                    'quarter': quarter,
                    'flow': flow,
                    'value': value
                })
            break

overall_trade_df = pd.DataFrame(overall_data)
print(f"📊 Extracted {len(overall_trade_df)} overall trade records")

📅 Found 9 quarters: ['2023Q1', '2023Q2', '2023Q3', '2023Q4', '2024Q1', '2024Q2', '2024Q3', '2024Q4', '2025Q1']
📊 Extracted 45 overall trade records


In [5]:
# Visualize overall trade trends
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Trade Volume Trends (US$ millions)', 'Trade Balance Trend'),
    specs=[[{"secondary_y": False}], [{"secondary_y": False}]]
)

# Trade volumes
for flow in ['Exports', 'Imports', 'Re-Exports']:
    flow_data = overall_trade_df[overall_trade_df['flow'] == flow]
    fig.add_trace(
        go.Scatter(
            x=flow_data['quarter'],
            y=flow_data['value'],
            mode='lines+markers',
            name=flow,
            line=dict(width=3)
        ),
        row=1, col=1
    )

# Trade balance
balance_data = overall_trade_df[overall_trade_df['flow'] == 'Trade Balance']
fig.add_trace(
    go.Bar(
        x=balance_data['quarter'],
        y=balance_data['value'],
        name='Trade Balance',
        marker_color='red'
    ),
    row=2, col=1
)

fig.update_layout(
    height=800,
    title_text="Rwanda's Overall Trade Performance (2023Q1 - 2025Q1)",
    showlegend=True
)

fig.show()

## 3. Export Destinations Analysis

Let's analyze the top export destinations from the 'ExportCountry' sheet.

In [6]:
# Load export country data
export_country_df = pd.read_excel(excel_file_path, sheet_name='ExportCountry', header=None)

# Extract country data
export_countries = []
quarters = []

# Find header row
for i, row in export_country_df.iterrows():
    if pd.notna(row.iloc[1]) and '2023Q1' in str(row.iloc[1]):
        quarters = [str(q) for q in row.iloc[1:-2] if pd.notna(q) and 'Q' in str(q)]
        break

print(f"📅 Export quarters found: {len(quarters)}")

# Extract country export data
start_row = None
for i, row in export_country_df.iterrows():
    if pd.notna(row.iloc[0]) and 'Total Estimates' in str(row.iloc[0]):
        start_row = i + 1
        break

if start_row:
    for i in range(start_row, len(export_country_df)):
        row = export_country_df.iloc[i]
        country = str(row.iloc[0]).strip() if pd.notna(row.iloc[0]) else None
        
        if not country or country in ['Source: NISR', '*Major']:
            continue
            
        # Extract values for each quarter
        for j, quarter in enumerate(quarters):
            col_index = j + 1
            if col_index < len(row):
                value = row.iloc[col_index]
                if pd.notna(value) and float(value) > 0:
                    export_countries.append({
                        'country': country,
                        'quarter': quarter,
                        'export_value': float(value)
                    })

export_countries_df = pd.DataFrame(export_countries)
print(f"🌍 Extracted {len(export_countries_df)} export country records")

📅 Export quarters found: 10
🌍 Extracted 199 export country records


In [7]:
# Top export destinations analysis
if not export_countries_df.empty:
    # Calculate total exports by country
    top_destinations = export_countries_df.groupby('country')['export_value'].sum().reset_index()
    top_destinations = top_destinations.nlargest(10, 'export_value').sort_values('export_value', ascending=True)
    
    # Create horizontal bar chart
    fig = px.bar(
        top_destinations,
        x='export_value',
        y='country',
        orientation='h',
        title='Top 10 Export Destinations (Total 2023Q1-2025Q1)',
        labels={'export_value': 'Export Value (US$ millions)', 'country': 'Destination Country'}
    )
    
    fig.update_layout(
        height=600,
        showlegend=False,
        margin=dict(l=150)
    )
    
    fig.show()
    
    print("💰 Top Export Destinations:")
    for _, row in top_destinations.iterrows():
        print(f"   {row['country']}: ${row['export_value']:,.2f} million")

💰 Top Export Destinations:
   Netherlands: $44.59 million
   Uganda: $56.83 million
   United States: $58.52 million
   Pakistan: $65.69 million
   Hong Kong: $75.22 million
   United Kingdom: $83.97 million
   Luxembourg: $86.02 million
   China: $196.09 million
   Congo, The Democratic Republic Of: $479.88 million
   United Arab Emirates: $2,819.37 million


## 4. Import Sources Analysis

Now let's analyze the import sources from the 'ImportCountry' sheet.

In [8]:
# Load import country data
import_country_df = pd.read_excel(excel_file_path, sheet_name='ImportCountry', header=None)

# Extract country data (similar structure to exports)
import_countries = []

# Find header row
for i, row in import_country_df.iterrows():
    if pd.notna(row.iloc[1]) and '2023Q1' in str(row.iloc[1]):
        quarters = [str(q) for q in row.iloc[1:-2] if pd.notna(q) and 'Q' in str(q)]
        break

print(f"📅 Import quarters found: {len(quarters)}")

# Extract country import data
start_row = None
for i, row in import_country_df.iterrows():
    if pd.notna(row.iloc[0]) and 'Total Estimates' in str(row.iloc[0]):
        start_row = i + 1
        break

if start_row:
    for i in range(start_row, len(import_country_df)):
        row = import_country_df.iloc[i]
        country = str(row.iloc[0]).strip() if pd.notna(row.iloc[0]) else None
        
        if not country or country in ['Source: NISR']:
            continue
            
        # Extract values for each quarter
        for j, quarter in enumerate(quarters):
            col_index = j + 1
            if col_index < len(row):
                value = row.iloc[col_index]
                if pd.notna(value) and float(value) > 0:
                    import_countries.append({
                        'country': country,
                        'quarter': quarter,
                        'import_value': float(value)
                    })

import_countries_df = pd.DataFrame(import_countries)
print(f"🌍 Extracted {len(import_countries_df)} import country records")

📅 Import quarters found: 10
🌍 Extracted 200 import country records


In [9]:
# Top import sources analysis
if not import_countries_df.empty:
    # Calculate total imports by country
    top_sources = import_countries_df.groupby('country')['import_value'].sum().reset_index()
    top_sources = top_sources.nlargest(10, 'import_value').sort_values('import_value', ascending=True)
    
    # Create horizontal bar chart
    fig = px.bar(
        top_sources,
        x='import_value',
        y='country',
        orientation='h',
        title='Top 10 Import Sources (Total 2023Q1-2025Q1)',
        labels={'import_value': 'Import Value (US$ millions)', 'country': 'Source Country'}
    )
    
    fig.update_layout(
        height=600,
        showlegend=False,
        margin=dict(l=150)
    )
    
    fig.show()
    
    print("📦 Top Import Sources:")
    for _, row in top_sources.iterrows():
        print(f"   {row['country']}: ${row['import_value']:,.2f} million")

📦 Top Import Sources:
   Malaysia: $256.91 million
   Turkey: $276.32 million
   South Africa: $314.81 million
   Uganda: $545.99 million
   Cameroon: $650.57 million
   United Arab Emirates: $802.31 million
   India: $1,276.84 million
   Kenya: $1,346.91 million
   Tanzania, United Republic Of: $1,901.45 million
   China: $2,703.23 million


## 5. Commodity Analysis

Let's analyze the trade data by commodity type using the commodity sheets.

In [10]:
# Load commodity data
export_commodity_df = pd.read_excel(excel_file_path, sheet_name='ExportsCommodity', header=None)
import_commodity_df = pd.read_excel(excel_file_path, sheet_name='ImportsCommodity', header=None)

# Extract commodity data
commodity_exports = []
commodity_imports = []

# Find quarters for commodities
for i, row in export_commodity_df.iterrows():
    if pd.notna(row.iloc[2]) and '2023Q1' in str(row.iloc[2]):
        quarters = [str(q) for q in row.iloc[2:-2] if pd.notna(q) and 'Q' in str(q)]
        break

print(f"📅 Commodity quarters found: {len(quarters)}")

# Extract export commodities
for i in range(4, len(export_commodity_df)):
    row = export_commodity_df.iloc[i]
    sitc_code = str(row.iloc[0]).strip() if pd.notna(row.iloc[0]) else None
    commodity = str(row.iloc[1]).strip() if pd.notna(row.iloc[1]) else None
    
    if not commodity or commodity in ['Source: NISR'] or sitc_code in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']:
        continue
        
    for j, quarter in enumerate(quarters):
        col_index = j + 2
        if col_index < len(row):
            value = row.iloc[col_index]
            if pd.notna(value) and float(value) > 0:
                commodity_exports.append({
                    'sitc_section': sitc_code,
                    'commodity': commodity,
                    'quarter': quarter,
                    'export_value': float(value)
                })

# Extract import commodities (similar logic)
for i in range(4, len(import_commodity_df)):
    row = import_commodity_df.iloc[i]
    sitc_code = str(row.iloc[0]).strip() if pd.notna(row.iloc[0]) else None
    commodity = str(row.iloc[1]).strip() if pd.notna(row.iloc[1]) else None
    
    if not commodity or commodity in ['Source: NISR'] or sitc_code in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']:
        continue
        
    for j, quarter in enumerate(quarters):
        col_index = j + 2
        if col_index < len(row):
            value = row.iloc[col_index]
            if pd.notna(value) and float(value) > 0:
                commodity_imports.append({
                    'sitc_section': sitc_code,
                    'commodity': commodity,
                    'quarter': quarter,
                    'import_value': float(value)
                })

commodity_exports_df = pd.DataFrame(commodity_exports)
commodity_imports_df = pd.DataFrame(commodity_imports)

print(f"📦 Extracted {len(commodity_exports_df)} export commodity records")
print(f"📦 Extracted {len(commodity_imports_df)} import commodity records")

📅 Commodity quarters found: 10
📦 Extracted 10 export commodity records
📦 Extracted 10 import commodity records


In [11]:
# Commodity analysis visualization
if not commodity_exports_df.empty:
    # Group by SITC section
    sitc_exports = commodity_exports_df.groupby('sitc_section')['export_value'].sum().reset_index()
    sitc_exports = sitc_exports.sort_values('export_value', ascending=False)
    
    # Create pie chart for export commodities by SITC section
    fig = px.pie(
        sitc_exports,
        values='export_value',
        names='sitc_section',
        title='Export Distribution by SITC Section (2023Q1-2025Q1)'
    )
    
    fig.update_traces(textposition='inside', textinfo='percent+label')
    fig.update_layout(height=600)
    
    fig.show()
    
    print("📊 Export Distribution by SITC Section:")
    for _, row in sitc_exports.iterrows():
        print(f"   Section {row['sitc_section']}: ${row['export_value']:,.2f} million")

📊 Export Distribution by SITC Section:
   Section SITC SECTION: $4,568.63 million


## 6. Trade Balance Analysis

Let's analyze the trade balance trends and identify key patterns.

In [12]:
# Calculate trade balance by quarter
quarterly_balance = []

for quarter in quarters:
    # Get export and import totals for this quarter
    export_total = overall_trade_df[
        (overall_trade_df['quarter'] == quarter) & 
        (overall_trade_df['flow'] == 'Exports')
    ]['value'].sum()
    
    import_total = overall_trade_df[
        (overall_trade_df['quarter'] == quarter) & 
        (overall_trade_df['flow'] == 'Imports')
    ]['value'].sum()
    
    balance = export_total - import_total
    
    quarterly_balance.append({
        'quarter': quarter,
        'exports': export_total,
        'imports': import_total,
        'balance': balance,
        'balance_type': 'Surplus' if balance > 0 else 'Deficit'
    })

balance_df = pd.DataFrame(quarterly_balance)

print("⚖️ Trade Balance Analysis:")
for _, row in balance_df.iterrows():
    print(f"   {row['quarter']}: ${row['balance']:,.2f} ({row['balance_type']})")

⚖️ Trade Balance Analysis:
   2023Q1: $-1,074.37 (Deficit)
   2023Q2: $-1,086.36 (Deficit)
   2023Q3: $-1,193.70 (Deficit)
   2023Q4: $-1,087.83 (Deficit)
   2024Q1: $-978.91 (Deficit)
   2024Q2: $-1,031.33 (Deficit)
   2024Q3: $-1,084.56 (Deficit)
   2024Q4: $-951.94 (Deficit)
   2025Q1: $-898.23 (Deficit)
   Shares in % Q1: $0.00 (Deficit)


In [13]:
# Trade balance visualization
fig = go.Figure()

# Add export and import lines
fig.add_trace(go.Scatter(
    x=balance_df['quarter'],
    y=balance_df['exports'],
    mode='lines+markers',
    name='Exports',
    line=dict(color='green', width=3)
))

fig.add_trace(go.Scatter(
    x=balance_df['quarter'],
    y=balance_df['imports'],
    mode='lines+markers',
    name='Imports',
    line=dict(color='red', width=3)
))

# Add balance as bars
colors = ['green' if x > 0 else 'red' for x in balance_df['balance']]
fig.add_trace(go.Bar(
    x=balance_df['quarter'],
    y=balance_df['balance'],
    name='Trade Balance',
    marker_color=colors,
    opacity=0.6
))

fig.update_layout(
    title='Rwanda Trade Balance Analysis (2023Q1-2025Q1)',
    xaxis_title='Quarter',
    yaxis_title='Value (US$ millions)',
    height=600,
    showlegend=True
)

fig.show()

## 7. Regional Trade Analysis

Let's analyze trade with different regional blocs and continents.

In [14]:
# Load regional trade data
regional_df = pd.read_excel(excel_file_path, sheet_name='Regional blocks', header=None)

# Extract regional data
regional_trade = []

# Find quarters for regional data
for i, row in regional_df.iterrows():
    if pd.notna(row.iloc[2]) and '2023Q1' in str(row.iloc[2]):
        quarters = [str(q) for q in row.iloc[2:] if pd.notna(q) and 'Q' in str(q)]
        break

print(f"📅 Regional quarters found: {len(quarters)}")

# Extract regional bloc data
for i in range(3, len(regional_df)):
    row = regional_df.iloc[i]
    bloc = str(row.iloc[0]).strip() if pd.notna(row.iloc[0]) else None
    flow = str(row.iloc[1]).strip() if pd.notna(row.iloc[1]) else None
    
    if not bloc or not flow or bloc in ['Source: NISR', 'Partner']:
        continue
        
    for j, quarter in enumerate(quarters):
        col_index = j + 2
        if col_index < len(row):
            value = row.iloc[col_index]
            if pd.notna(value) and float(value) > 0:
                regional_trade.append({
                    'regional_bloc': bloc,
                    'flow': flow,
                    'quarter': quarter,
                    'value': float(value)
                })

regional_trade_df = pd.DataFrame(regional_trade)
print(f"🌍 Extracted {len(regional_trade_df)} regional trade records")

📅 Regional quarters found: 9
🌍 Extracted 54 regional trade records


In [15]:
# Regional trade analysis
if not regional_trade_df.empty:
    # Group by regional bloc and flow
    regional_summary = regional_trade_df.groupby(['regional_bloc', 'flow'])['value'].sum().reset_index()
    
    # Create grouped bar chart
    fig = px.bar(
        regional_summary,
        x='regional_bloc',
        y='value',
        color='flow',
        barmode='group',
        title='Trade by Regional Blocs (Total 2023Q1-2025Q1)',
        labels={'value': 'Trade Value (US$ millions)', 'regional_bloc': 'Regional Bloc'}
    )
    
    fig.update_layout(
        height=600,
        xaxis_tickangle=-45
    )
    
    fig.show()
    
    print("🌍 Regional Trade Summary:")
    for _, row in regional_summary.iterrows():
        print(f"   {row['regional_bloc']} - {row['flow']}: ${row['value']:,.2f} million")

🌍 Regional Trade Summary:
   CEPGL - Export: $497.64 million
   COMESA - Export: $627.20 million
   COMMON WEALTH - Export: $317.49 million
   ECOWAS - Export: $2.91 million
   EU - Export: $260.76 million
   SADC - Export: $490.68 million


## 8. EAC Trade Analysis

Let's analyze trade specifically with East African Community countries.

In [16]:
# Load EAC trade data
excel_file_path = "../data/raw/2025Q1_Trade_report_annexTables.xlsx"
eac_df = pd.read_excel(excel_file_path, sheet_name='EAC', header=None)

# Extract EAC data
eac_trade = []

print(f"📅 Using quarters from overall trade analysis: {len(quarters)}")

# Extract EAC country data
for i in range(4, len(eac_df)):
    row = eac_df.iloc[i]
    flow = str(row.iloc[0]).strip() if pd.notna(row.iloc[0]) else None
    country = str(row.iloc[1]).strip() if pd.notna(row.iloc[1]) else None
    
    if not flow or not country or country in ['Source: NISR', 'EAC'] or flow in ['Re-Exports']:
        continue
        
    for j, quarter in enumerate(quarters):
        col_index = j + 2
        if col_index < len(row):
            value = row.iloc[col_index]
            if pd.notna(value) and float(value) > 0:
                eac_trade.append({
                    'country': country,
                    'quarter': quarter,
                    'value': float(value)
                })

eac_trade_df = pd.DataFrame(eac_trade)
print(f"🌍 Extracted {len(eac_trade_df)} EAC trade records")

📅 Using quarters from overall trade analysis: 9
🌍 Extracted 0 EAC trade records


In [17]:
# EAC trade analysis
if not eac_trade_df.empty:
    # Group by country and flow
    eac_summary = eac_trade_df.groupby(['country', 'flow'])['value'].sum().reset_index()
    
    # Create heatmap-style visualization
    eac_pivot = eac_summary.pivot(index='country', columns='flow', values='value').fillna(0)
    
    fig = go.Figure(data=go.Heatmap(
        z=eac_pivot.values,
        x=eac_pivot.columns,
        y=eac_pivot.index,
        colorscale='Blues',
        text=np.round(eac_pivot.values, 2),
        texttemplate='%{text}',
        textfont={"size":10},
        colorbar=dict(title="Trade Value (US$ millions)")
    ))
    
    fig.update_layout(
        title='EAC Trade Matrix (Total 2023Q1-2025Q1)',
        xaxis_title='Trade Flow',
        yaxis_title='EAC Country',
        height=600
    )
    
    fig.show()
    
    print("🌍 EAC Trade Summary:")
    for _, row in eac_summary.iterrows():
        print(f"   {row['country']} - {row['flow']}: ${row['value']:,.2f} million")

## 9. Continental Trade Analysis

Let's analyze trade by continents.

In [18]:
# Load continental trade data
try:
    continental_df = pd.read_excel("../data/raw/2025Q1_Trade_report_annexTables.xlsx", 
                                 sheet_name='Trade by continents', 
                                 header=None)
    
    # Extract continental data
    continental_trade = []

    # Find quarters for continental data
    for i, row in continental_df.iterrows():
        if pd.notna(row.iloc[2]) and '2023Q1' in str(row.iloc[2]):
            cont_quarters = [str(q) for q in row.iloc[2:11] if pd.notna(q) and 'Q' in str(q)]
            break

    print(f"📅 Continental quarters found: {len(cont_quarters)}")

    # Extract continental data
    for i in range(3, len(continental_df)):
        row = continental_df.iloc[i]
        flow = str(row.iloc[0]).strip() if pd.notna(row.iloc[0]) else None
        continent = str(row.iloc[1]).strip() if pd.notna(row.iloc[1]) else None
        
        if not flow or not continent or continent in ['WORLD', 'Source: NISR']:
            continue
            
        for j, quarter in enumerate(cont_quarters):
            col_index = j + 2
            if col_index < len(row):
                value = row.iloc[col_index]
                if pd.notna(value) and float(value) > 0:
                    continental_trade.append({
                        'flow': flow,
                        'continent': continent,
                        'quarter': quarter,
                        'value': float(value)
                    })

    continental_trade_df = pd.DataFrame(continental_trade)
    print(f"🌍 Extracted {len(continental_trade_df)} continental trade records")
    
except Exception as e:
    print(f"❌ Error loading continental trade data: {e}")
    continental_trade_df = pd.DataFrame()  # Create empty DataFrame if loading fails

# Extract continental data
continental_trade = []

# Find quarters for continental data
for i, row in continental_df.iterrows():
    if pd.notna(row.iloc[2]) and '2023Q1' in str(row.iloc[2]):
        cont_quarters = [str(q) for q in row.iloc[2:11] if pd.notna(q) and 'Q' in str(q)]
        break

print(f"📅 Continental quarters found: {len(cont_quarters)}")

# Extract continental data
for i in range(3, len(continental_df)):
    row = continental_df.iloc[i]
    flow = str(row.iloc[0]).strip() if pd.notna(row.iloc[0]) else None
    continent = str(row.iloc[1]).strip() if pd.notna(row.iloc[1]) else None
    
    if not flow or not continent or continent in ['WORLD', 'Source: NISR']:
        continue
        
    for j, quarter in enumerate(cont_quarters):
        col_index = j + 2
        if col_index < len(row):
            value = row.iloc[col_index]
            if pd.notna(value) and float(value) > 0:
                continental_trade.append({
                    'flow': flow,
                    'continent': continent,
                    'quarter': quarter,
                    'value': float(value)
                })

continental_trade_df = pd.DataFrame(continental_trade)
print(f"🌍 Extracted {len(continental_trade_df)} continental trade records")

📅 Continental quarters found: 9
❌ Error loading continental trade data: could not convert string to float: '2023Q1'
📅 Continental quarters found: 9


ValueError: could not convert string to float: '2023Q1'

In [ ]:
# Continental trade analysis
if not continental_trade_df.empty:
    # Group by continent and flow
    continental_summary = continental_trade_df.groupby(['continent', 'flow'])['value'].sum().reset_index()
    
    # Create grouped bar chart
    fig = px.bar(
        continental_summary,
        x='continent',
        y='value',
        color='flow',
        barmode='group',
        title='Trade by Continent (Total 2023Q1-2025Q1)',
        labels={'value': 'Trade Value (US$ millions)', 'continent': 'Continent'}
    )
    
    fig.update_layout(
        height=600,
        xaxis_tickangle=-45
    )
    
    fig.show()
    
    print("🌍 Continental Trade Summary:")
    for _, row in continental_summary.iterrows():
        print(f"   {row['continent']} - {row['flow']}: ${row['value']:,.2f} million")

## 10. Growth Analysis

Let's analyze growth trends and patterns in Rwanda's trade.

In [ ]:
# Calculate growth rates
growth_analysis = []

for flow in ['Exports', 'Imports']:
    flow_data = overall_trade_df[overall_trade_df['flow'] == flow].copy()
    flow_data = flow_data.sort_values('quarter')
    
    for i in range(1, len(flow_data)):
        current = flow_data.iloc[i]
        previous = flow_data.iloc[i-1]
        
        growth_rate = ((current['value'] - previous['value']) / previous['value']) * 100
        
        growth_analysis.append({
            'flow': flow,
            'quarter': current['quarter'],
            'value': current['value'],
            'growth_rate': growth_rate,
            'growth_type': 'Growth' if growth_rate > 0 else 'Decline'
        })

growth_df = pd.DataFrame(growth_analysis)

print("📈 Growth Analysis Summary:")
for _, row in growth_df.iterrows():
    print(f"   {row['flow']} {row['quarter']}: {row['growth_rate']:+.2f}% ({row['growth_type']})")

In [ ]:
# Growth visualization
fig = go.Figure()

for flow in ['Exports', 'Imports']:
    flow_growth = growth_df[growth_df['flow'] == flow]
    
    fig.add_trace(go.Bar(
        x=flow_growth['quarter'],
        y=flow_growth['growth_rate'],
        name=flow,
        marker_color='green' if flow == 'Exports' else 'red'
    ))

fig.update_layout(
    title='Quarter-over-Quarter Growth Rates',
    xaxis_title='Quarter',
    yaxis_title='Growth Rate (%)',
    height=500,
    showlegend=True
)

fig.show()

## 11. Key Insights and Summary

Let's compile the key insights from our comprehensive analysis.

In [ ]:
# Generate comprehensive summary
summary = {
    'dataset_overview': {
        'period_covered': f"{quarters[0]} to {quarters[-1]}",
        'total_quarters': len(quarters),
        'sheets_analyzed': len(sheet_names)
    },
    'trade_summary': {
        'total_exports': balance_df['exports'].sum(),
        'total_imports': balance_df['imports'].sum(),
        'overall_balance': balance_df['balance'].sum(),
        'balance_status': 'Deficit' if balance_df['balance'].sum() < 0 else 'Surplus'
    },
    'top_performers': {
        'top_export_destination': top_destinations.iloc[-1]['country'] if 'top_destinations' in locals() else 'N/A',
        'top_import_source': top_sources.iloc[-1]['country'] if 'top_sources' in locals() else 'N/A',
        'dominant_continent_export': continental_summary[continental_summary['flow']=='Exports'].nlargest(1, 'value')['continent'].iloc[0] if 'continental_summary' in locals() else 'N/A',
        'dominant_continent_import': continental_summary[continental_summary['flow']=='Imports'].nlargest(1, 'value')['continent'].iloc[0] if 'continental_summary' in locals() else 'N/A'
    },
    'growth_trends': {
        'avg_export_growth': growth_df[growth_df['flow']=='Exports']['growth_rate'].mean(),
        'avg_import_growth': growth_df[growth_df['flow']=='Imports']['growth_rate'].mean(),
        'export_volatility': growth_df[growth_df['flow']=='Exports']['growth_rate'].std(),
        'import_volatility': growth_df[growth_df['flow']=='Imports']['growth_rate'].std()
    }
}

print("📊 COMPREHENSIVE TRADE ANALYSIS SUMMARY")
print("="*60)
print(f"📅 Period Covered: {summary['dataset_overview']['period_covered']}")
print(f"📋 Quarters Analyzed: {summary['dataset_overview']['total_quarters']}")
print(f"📊 Sheets Processed: {summary['dataset_overview']['sheets_analyzed']}")
print()
print("💰 Trade Summary:")
print(f"   Total Exports: ${summary['trade_summary']['total_exports']:,.2f} million")
print(f"   Total Imports: ${summary['trade_summary']['total_imports']:,.2f} million")
print(f"   Overall Balance: ${summary['trade_summary']['overall_balance']:,.2f} million ({summary['trade_summary']['balance_status']})")
print()
print("🏆 Top Performers:")
print(f"   Top Export Destination: {summary['top_performers']['top_export_destination']}")
print(f"   Top Import Source: {summary['top_performers']['top_import_source']}")
print(f"   Main Export Continent: {summary['top_performers']['dominant_continent_export']}")
print(f"   Main Import Continent: {summary['top_performers']['dominant_continent_import']}")
print()
print("📈 Growth Trends:")
print(f"   Average Export Growth: {summary['growth_trends']['avg_export_growth']:+.2f}%")
print(f"   Average Import Growth: {summary['growth_trends']['avg_import_growth']:+.2f}%")
print(f"   Export Volatility: {summary['growth_trends']['export_volatility']:.2f}%")
print(f"   Import Volatility: {summary['growth_trends']['import_volatility']:.2f}%")
print("="*60)

NameError: name 'growth_df' is not defined

## 12. Export Analysis Results to JSON

Let's save our comprehensive analysis results to JSON files for use in the backend.

In [ ]:
# Create comprehensive analysis results
analysis_results = {
    'metadata': {
        'analysis_date': pd.Timestamp.now().isoformat(),
        'data_source': '2025Q1_Trade_report_annexTables.xlsx',
        'period_covered': quarters,
        'analysis_type': 'comprehensive_trade_analysis'
    },
    'overall_trade': overall_trade_df.to_dict('records'),
    'export_countries': export_countries_df.to_dict('records') if 'export_countries_df' in locals() else [],
    'import_countries': import_countries_df.to_dict('records') if 'import_countries_df' in locals() else [],
    'commodity_exports': commodity_exports_df.to_dict('records') if 'commodity_exports_df' in locals() else [],
    'commodity_imports': commodity_imports_df.to_dict('records') if 'commodity_imports_df' in locals() else [],
    'regional_trade': regional_trade_df.to_dict('records') if 'regional_trade_df' in locals() else [],
    'eac_trade': eac_trade_df.to_dict('records') if 'eac_trade_df' in locals() else [],
    'continental_trade': continental_trade_df.to_dict('records') if 'continental_trade_df' in locals() else [],
    'trade_balance': balance_df.to_dict('records'),
    'growth_analysis': growth_df.to_dict('records'),
    'summary': summary
}

# Save to JSON files
output_dir = Path("../data/processed")
output_dir.mkdir(parents=True, exist_ok=True)

# Save main analysis results
with open(output_dir / 'jupyter_analysis_results.json', 'w', encoding='utf-8') as f:
    json.dump(analysis_results, f, indent=2, ensure_ascii=False, default=str)

print(f"✅ Analysis results saved to: {output_dir / 'jupyter_analysis_results.json'}")

# Save summary for quick access
with open(output_dir / 'analysis_summary.json', 'w', encoding='utf-8') as f:
    json.dump(summary, f, indent=2, ensure_ascii=False, default=str)

print(f"✅ Summary saved to: {output_dir / 'analysis_summary.json'}")

print("\n🎉 Comprehensive analysis complete! All results saved to JSON files.")